In [1]:
import shapefile as shp

In [3]:
precincts = shp.Reader("../shapefiles/police_precincts/nypp")

In [4]:
tracts = shp.Reader("../shapefiles/census_tracts/nyct2010")

In [6]:
len(precincts.shapes())

77

In [7]:
len(tracts.shapes())

2166